## Scraping Holiday (ไว้โม้ว่าเขาไใ่ทำงานวันหยุด info to llm)

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

HOLIDAY_URL = "https://www.myhora.com/calendar/ical/holiday.aspx?latest.txt"

def convert_thai_date_to_gregorian(thai_date_str):
    """
    Converts Thai Buddhist date string (e.g., '01 ม.ค. 2567') to ISO or Redis-friendly format (dd/mm/yyyy).
    """
    thai_months = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }

    match = re.match(r'(\d{1,2})\s(\S+)\s(\d{4})', thai_date_str.strip())
    if not match:
        return None

    day, thai_month, thai_year = match.groups()
    month = thai_months.get(thai_month)
    year = int(thai_year) - 543  # Convert BE to CE

    return f"{int(day):02d}/{month}/{year}"


def scrape_myhora_holidays(url=HOLIDAY_URL):
    response = requests.get(url)
    response.encoding = "utf-8"  # Ensure Thai characters are correctly parsed

    if response.status_code != 200:
        raise Exception(f"Failed to fetch data. Status code: {response.status_code}")

    soup = BeautifulSoup(response.text, "html.parser")
    holiday_divs = soup.find_all("div", class_="mb-5")

    holidays = []
    for div in holiday_divs:
        columns = div.find_all("div")
        if len(columns) >= 2:
            raw_date = columns[0].text.strip()
            name = columns[1].text.strip()
            formatted_date = convert_thai_date_to_gregorian(raw_date)

            if formatted_date:
                holidays.append({
                    "date": formatted_date,
                    "name": name
                })

    return holidays


if __name__ == "__main__":
    holidays = scrape_myhora_holidays()
    for h in holidays:
        print(h)


{'date': '01/01/2024', 'name': 'วันขึ้นปีใหม่'}
{'date': '24/02/2024', 'name': 'วันมาฆบูชา'}
{'date': '26/02/2024', 'name': 'วันหยุดชดเชย'}
{'date': '06/04/2024', 'name': 'วันจักรี'}
{'date': '08/04/2024', 'name': 'วันหยุดชดเชย'}
{'date': '12/04/2024', 'name': 'วันหยุดพิเศษ (ครม.)'}
{'date': '13/04/2024', 'name': 'วันสงกรานต์'}
{'date': '14/04/2024', 'name': 'วันสงกรานต์'}
{'date': '15/04/2024', 'name': 'วันสงกรานต์'}
{'date': '16/04/2024', 'name': 'วันหยุดชดเชย'}
{'date': '01/05/2024', 'name': 'วันแรงงาน (เอกชน)'}
{'date': '04/05/2024', 'name': 'วันฉัตรมงคล'}
{'date': '06/05/2024', 'name': 'วันหยุดชดเชย'}
{'date': '10/05/2024', 'name': 'วันพืชมงคล (ราชการ)'}
{'date': '22/05/2024', 'name': 'วันวิสาขบูชา'}
{'date': '03/06/2024', 'name': 'วันเฉลิมฯ พระบรมราชินี'}
{'date': '20/07/2024', 'name': 'วันอาสาฬหบูชา'}
{'date': '21/07/2024', 'name': 'วันเข้าพรรษา (ราชการ)'}
{'date': '22/07/2024', 'name': 'วันหยุดชดเชย'}
{'date': '28/07/2024', 'name': 'วันเฉลิมฯ พระวชิรเกล้าเจ้าอยู่หัว'}
{'date': 